In [5]:
import os
os.chdir(r'E:\ucc\project_sjfan')

import pandas as pd
df1 = pd.read_excel('data/globalterrorismdb_0522dist.xlsx')
df2 = pd.read_excel('data/globalterrorismdb_2021Jan-June_1222dist.xlsx')

In [6]:
print("1970-2020 basic info:")
print(df1.info())
print("\nJan-June 2021 basic info:")
print(df2.info())

1970-2020 basic info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209706 entries, 0 to 209705
Columns: 135 entries, eventid to related
dtypes: datetime64[ns](1), float64(54), int64(23), object(57)
memory usage: 216.0+ MB
None

Jan-June 2021 basic info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4960 entries, 0 to 4959
Columns: 135 entries, eventid to related
dtypes: datetime64[ns](1), float64(51), int64(29), object(54)
memory usage: 5.1+ MB
None


In [3]:
print("\n1970-2020 数据预览:")
print(df1.head())
print("\n2021年1-6月 数据预览:")
print(df2.head())


1970-2020 数据预览:
        eventid  iyear  imonth  iday approxdate  extended resolution  country  \
0  197000000001   1970       7     2        NaN         0        NaT       58   
1  197000000002   1970       0     0        NaN         0        NaT      130   
2  197001000001   1970       1     0        NaN         0        NaT      160   
3  197001000002   1970       1     0        NaN         0        NaT       78   
4  197001000003   1970       1     0        NaN         0        NaT      101   

          country_txt  region  ... addnotes scite1 scite2  scite3  dbsource  \
0  Dominican Republic       2  ...      NaN    NaN    NaN     NaN      PGIS   
1              Mexico       1  ...      NaN    NaN    NaN     NaN      PGIS   
2         Philippines       5  ...      NaN    NaN    NaN     NaN      PGIS   
3              Greece       8  ...      NaN    NaN    NaN     NaN      PGIS   
4               Japan       4  ...      NaN    NaN    NaN     NaN      PGIS   

   INT_LOG  INT_IDEO 

In [11]:
merge_df = pd.concat([df1,df2])
data = merge_df[['iyear','country_txt','region_txt','provstate','city','latitude','longitude','attacktype1',
                 'attacktype1_txt','targtype1_txt','nkill','nwound']]
data = data.rename(columns={'iyear':'Year','country_txt':'Country','region_txt':'Region','provstate':'state',
                       'attacktype1':'AttackTypeCode','attacktype1_txt':'AttackType','targtype1_txt':'Target','nkill':'Killed',
                       'nwound':'Wounded'})
csv_file = 'data/merged_global_terrorism_database.csv'
data.to_csv(csv_file, index=False)

data = pd.read_csv('data/merged_global_terrorism_database.csv')
data.shape

(214666, 12)

后续可以直接读取csv代码节省时间

In [8]:
data = pd.read_csv('data/merged_global_terrorism_database.csv')
data.shape

(214666, 12)

In [12]:
print(data.columns)

Index(['Year', 'Country', 'Region', 'state', 'city', 'latitude', 'longitude',
       'AttackTypeCode', 'AttackType', 'Target', 'Killed', 'Wounded'],
      dtype='object')


In [13]:
print(data.isnull().sum())

Year                  0
Country               0
Region                0
state                 0
city                427
latitude           4726
longitude          4727
AttackTypeCode        0
AttackType            0
Target                0
Killed            12951
Wounded           20705
dtype: int64


In [14]:
data[['Killed','Wounded']] = data[['Killed','Wounded']].fillna(0)

In [18]:
duplicate_count = data.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

data = data.drop_duplicates()

Number of duplicate rows: 37282


In [15]:
data.isnull().sum()

Year                 0
Country              0
Region               0
state                0
city               427
latitude          4726
longitude         4727
AttackTypeCode       0
AttackType           0
Target               0
Killed               0
Wounded              0
dtype: int64

In [116]:
year_min,year_max = data['Year'].min(),data['Year'].max()

region_ls = data['Region'].unique().tolist()
country_ls = data['Country'].unique().tolist()

In [120]:
df_pie = data.groupby(['Region','Country','Year','AttackType'])[['Killed','Wounded']].sum().reset_index()
attacktype_count = data.groupby(['Region', 'Country', 'Year','AttackType']).size().reset_index(name='Count')
df_pie = pd.merge(df_pie, attacktype_count, on=['Region', 'Country', 'Year','AttackType'])
df_pie

,Region,Country,Year,AttackType,Killed,Wounded,Count
0,Australasia & Oceania,Australia,1970,Bombing/Explosion,0.0,0.0,1
1,Australasia & Oceania,Australia,1971,Bombing/Explosion,0.0,0.0,1
2,Australasia & Oceania,Australia,1972,Assassination,0.0,0.0,6
3,Australasia & Oceania,Australia,1972,Bombing/Explosion,1.0,0.0,2
4,Australasia & Oceania,Australia,1974,Facility/Infrastructure Attack,0.0,0.0,1
...,...,...,...,...,...,...,...
12773,Western Europe,West Germany (FRG),1989,Bombing/Explosion,0.0,2.0,6
12774,Western Europe,West Germany (FRG),1989,Facility/Infrastructure Attack,0.0,0.0,9
12775,Western Europe,West Germany (FRG),1990,Assassination,1.0,1.0,2
12776,Western Europe,West Germany (FRG),1990,Bombing/Explosion,0.0,3.0,5


In [130]:
def get_pie_data(region,country,span_year):
    df_filter = df_pie[
        (df_pie['Region']==region) & (df_pie['Country']==country) & (df_pie['Year']>=span_year[0]) & (df_pie['Year']<=span_year[1])
    ]
    pie_data = df_filter.groupby('AttackType')['Count'].sum().reset_index()
    return pie_data

In [114]:
df_bar = data.groupby(['Region','Country','Year'])[['Killed','Wounded']].sum().reset_index()
attack_count = data.groupby(['Region', 'Country', 'Year']).size().reset_index(name='AttackCount')
df_bar = pd.merge(df_bar, attack_count, on=['Region', 'Country', 'Year'])

In [104]:
df_bar

,Region,Country,Year,Killed,Wounded,AttackCount
0,Australasia & Oceania,Australia,1970,0.0,0.0,1
1,Australasia & Oceania,Australia,1971,0.0,0.0,1
2,Australasia & Oceania,Australia,1972,1.0,0.0,8
3,Australasia & Oceania,Australia,1974,0.0,0.0,1
4,Australasia & Oceania,Australia,1978,2.0,9.0,2
...,...,...,...,...,...,...
4144,Western Europe,West Germany (FRG),1986,9.0,242.0,46
4145,Western Europe,West Germany (FRG),1987,2.0,32.0,19
4146,Western Europe,West Germany (FRG),1988,1.0,14.0,14
4147,Western Europe,West Germany (FRG),1989,5.0,8.0,21


In [70]:
def get_bar_data(region,country,span_year):

    df_filter = df_bar[
        (df_bar['Region']==region) & (df_bar['Country']==country) & (df_bar['Year']>=span_year[0]) & (df_bar['Year']<=span_year[1])
    ]
    filter_data_ls = df_filter[['Killed','Wounded','AttackCount']].values.T.tolist()
    return filter_data_ls

In [79]:
geo_df = data.groupby(['Region','Country','Year','latitude','longitude'])[['Killed','Wounded']].sum().reset_index()
attack_types = data.groupby(['Region','Country','Year','latitude','longitude'])['AttackType'].apply(list).reset_index(name='AttackTypes')
attack_count = data.groupby(['Region', 'Country', 'Year','latitude','longitude']).size().reset_index(name='AttackCount')
geo_df = pd.merge(geo_df, attack_types, on=['Region','Country','Year','latitude','longitude'])
geo_df = pd.merge(geo_df,attack_count,on=['Region','Country','Year','latitude','longitude'])
geo_df

,Region,Country,Year,latitude,longitude,Killed,Wounded,AttackTypes,AttackCount
0,Australasia & Oceania,Australia,1970,-37.813187,144.962980,0.0,0.0,[Bombing/Explosion],1
1,Australasia & Oceania,Australia,1971,-33.873651,151.206890,0.0,0.0,[Bombing/Explosion],1
2,Australasia & Oceania,Australia,1972,-33.868820,151.209295,0.0,0.0,[Assassination],1
3,Australasia & Oceania,Australia,1972,-27.470933,153.023502,1.0,0.0,"[Bombing/Explosion, Bombing/Explosion]",2
4,Australasia & Oceania,Australia,1974,-33.873651,151.206890,0.0,0.0,[Facility/Infrastructure Attack],1
...,...,...,...,...,...,...,...,...,...
98316,Western Europe,West Germany (FRG),1990,51.512054,7.463573,1.0,0.0,[Assassination],1
98317,Western Europe,West Germany (FRG),1990,52.104026,9.357261,0.0,0.0,[Bombing/Explosion],1
98318,Western Europe,West Germany (FRG),1990,52.372068,9.735686,0.0,0.0,[Bombing/Explosion],1
98319,Western Europe,West Germany (FRG),1990,52.501530,13.401851,0.0,0.0,"[Facility/Infrastructure Attack, Facility/Infr...",2


In [82]:
option_df = data[['Region','Country']].groupby('Region').agg({'Country':lambda x:list(set(x))}).reset_index()
options_series = option_df.apply(lambda x: {x['Region']:x['Country']},axis=1)
all_options = {}
for i in options_series.values:
    all_options.update(i)

In [83]:
all_options

{'Australasia & Oceania': ['Wallis and Futuna',
  'New Caledonia',
  'New Zealand',
  'Papua New Guinea',
  'French Polynesia',
  'Vanuatu',
  'New Hebrides',
  'Fiji',
  'Australia',
  'Solomon Islands'],
 'Central America & Caribbean': ['St. Lucia',
  'St. Kitts and Nevis',
  'Cuba',
  'Dominican Republic',
  'Panama',
  'Honduras',
  'Haiti',
  'Barbados',
  'Nicaragua',
  'Belize',
  'Grenada',
  'Jamaica',
  'Antigua and Barbuda',
  'Guadeloupe',
  'Bahamas',
  'Dominica',
  'Guatemala',
  'Costa Rica',
  'Martinique',
  'El Salvador',
  'Trinidad and Tobago'],
 'Central Asia': ['Tajikistan',
  'Kazakhstan',
  'Turkmenistan',
  'Kyrgyzstan',
  'Azerbaijan',
  'Armenia',
  'Uzbekistan',
  'Georgia'],
 'East Asia': ['Japan',
  'North Korea',
  'Taiwan',
  'Hong Kong',
  'South Korea',
  'China',
  'Macau'],
 'Eastern Europe': ['Bosnia-Herzegovina',
  'Belarus',
  'Poland',
  'Estonia',
  'Serbia-Montenegro',
  'Macedonia',
  'Soviet Union',
  'Hungary',
  'Bulgaria',
  'Montenegro',